# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [26]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [27]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,1.43,58,99,7.79,TF,1688882248
1,1,grytviken,-54.2811,-36.5092,-5.58,83,70,5.42,GS,1688882249
2,2,swakopmund,-22.6833,14.5333,7.47,71,0,5.73,NaN,1688882250
3,3,bodo,67.2800,14.4050,11.08,88,75,2.06,NO,1688882251
4,4,sretensk,52.2453,117.7093,29.73,32,96,5.28,RU,1688882252


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    alpha = 0.5,
    color = "City"
    
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data = (city_data_df.loc[(city_data_df['Max Temp']>21)&
                                              (city_data_df['Max Temp']<27)&
                                              (city_data_df['Wind Speed']<4.5)&
                                              (city_data_df['Cloudiness']==0)])

# Drop any rows with null values
ideal_city_data = ideal_city_data.dropna()

# Display sample data
ideal_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
76,76,morteau,47.0667,6.6167,22.09,88,0,0.51,FR,1688882326
86,86,westport,41.1415,-73.3579,24.49,89,0,0.00,US,1688882112
100,100,alcala de henares,40.4821,-3.3600,22.91,45,0,2.06,ES,1688882350
121,121,carnarvon,-24.8667,113.6333,22.36,33,0,3.93,AU,1688882371
150,150,senj,44.9894,14.9058,26.49,61,0,2.07,HR,1688882401
171,171,colonia,50.9333,6.9500,24.90,67,0,1.03,DE,1688882450
191,191,hola,51.5377,23.1902,21.01,100,0,1.79,PL,1688882475
195,195,remire-montjoly,4.9167,-52.2667,24.02,94,0,0.00,GF,1688882480
200,200,porto tolle,44.9497,12.3245,23.80,57,0,1.31,IT,1688882486
217,217,lengnau,47.1838,7.3737,22.89,85,0,0.51,CH,1688882505


### Step 3: Create a new DataFrame called `hotel_df`.

In [75]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data[['City','Country','Humidity','Lat','Lng']]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "
# Display sample data
hotel_df

C:\Users\dyoxm\AppData\Local\Temp\ipykernel_60288\3624617828.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Country,Humidity,Lat,Lng,Hotel Name
76,morteau,FR,88,47.0667,6.6167,
86,westport,US,89,41.1415,-73.3579,
100,alcala de henares,ES,45,40.4821,-3.3600,
121,carnarvon,AU,33,-24.8667,113.6333,
150,senj,HR,61,44.9894,14.9058,
171,colonia,DE,67,50.9333,6.9500,
191,hola,PL,100,51.5377,23.1902,
195,remire-montjoly,GF,94,4.9167,-52.2667,
200,porto tolle,IT,57,44.9497,12.3245,
217,lengnau,CH,85,47.1838,7.3737,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [76]:
# Set parameters to search for a hotel
radius = 10000
category = "accommodation.hotel"
params = {
    "apiKey":geoapify_key,
    "categories":category
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = ideal_city_data['Lat'][index]
    Longitude = ideal_city_data['Lng'][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
morteau - nearest hotel: La Guimbarde
westport - nearest hotel: Norwalk Inn and Conference Center
alcala de henares - nearest hotel: Parador de Alcalá de Henares
carnarvon - nearest hotel: No hotel found
senj - nearest hotel: Laura apartmanban
colonia - nearest hotel: Wasserturm Hotel Cologne
hola - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
porto tolle - nearest hotel: albergo Italia
lengnau - nearest hotel: Passage
vila velha - nearest hotel: Hotel Prainha
zmigrod - nearest hotel: Hotelik BARYCZ
fallon - nearest hotel: Value Inn
fort bragg - nearest hotel: Airborne Inn Lodging
greenwood - nearest hotel: Hospitality Suites
essaouira - nearest hotel: Essaouira Wind Palace
dunwoody - nearest hotel: Residence Inn Atlanta Perimeter Center/Dunwoody
florida city - nearest hotel: Tru by Hilton Florida City
khanty-mansiysk - nearest hotel: Гамма
steinbach - nearest hotel: Alt Steinbach
turaif - nearest hotel: المودة
norfolk - nearest h

,City,Country,Humidity,Lat,Lng,Hotel Name
76,morteau,FR,88,47.0667,6.6167,La Guimbarde
86,westport,US,89,41.1415,-73.3579,Norwalk Inn and Conference Center
100,alcala de henares,ES,45,40.4821,-3.3600,Parador de Alcalá de Henares
121,carnarvon,AU,33,-24.8667,113.6333,No hotel found
150,senj,HR,61,44.9894,14.9058,Laura apartmanban
171,colonia,DE,67,50.9333,6.9500,Wasserturm Hotel Cologne
191,hola,PL,100,51.5377,23.1902,No hotel found
195,remire-montjoly,GF,94,4.9167,-52.2667,Complexe Belova
200,porto tolle,IT,57,44.9497,12.3245,albergo Italia
217,lengnau,CH,85,47.1838,7.3737,Passage


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [77]:
%%capture --no-display

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    alpha = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)